# IMPORTING LIBRARIES

In [1]:
import numpy as np #Biblioteca de suporte de arrays e matriz multidimensionais
import pandas as pd #Biblioteca de manipulação e análise de dado
import os #Esse modulo prove alguns metodos para atuar com o sistema operacional, nesse projeto utilizado para gerenciamento de diretorios
import sys #Esse modulo prove acesso à algumas variaveis de sistema, nesse projeto utilizado para gerenciamento de diretorios
import matplotlib.pyplot as plt #Biblioteca padrão pala plotagem de graficos
#import seaborn as sns #Biblioteca para melhoria de interface dos graficos
sys.path.append('../abstract_normalizer')
import dictionaries 
import json
#import operator
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
import math
import inflection
import dill

## DECLARING BASIC FUNCTIONS

In [2]:
#Function that removes files that have especified extensions from a list of files 'li'
def cleanDirectory(li):
    try:
        for directory in li:
            if(not directory.endswith(".json") ):
                li.remove(directory)
                cleanDirectory(li)
    except:
        print("")

#Function that gets all the fields, disciplines and journalsof the corpus
def getAllFields():
    return {
    "AGRONOMY", 
    "BUSINESS", 
    "CHEMISTRY", 
    "COGNITIVE LINGUISTICS",
    "COMPUTER SCIENCE",
    "ENGINEERING", 
    "HEALTH CARE SCIENCES & SERVICES", 
    "LINGUISTICS", 
    "PHYSICS", 
    "TELECOMMUNICATIONS"
    }

def getAllDisciplines():
    return {
            "BIOLOGICAL SCIENCES",
            "SOCIAL & HUMAN SCIENCES",
            "HARD SCIENCES"
    }

def getAllJournals():
    return {
    "ACADEMIC MEDICINE",
    "ACADEMY OF MANAGEMENT ANNALS" ,
    "ACADEMY OF MANAGEMENT REVIEW" ,
    "ADVANCED MATERIALS" ,
    "ADVANCES IN AGRONOMY, VOL 129",
    "ADVANCES IN AGRONOMY, VOL 130",
    "ADVANCES IN AGRONOMY, VOL 131",
    "ADVANCES IN AGRONOMY, VOL 132",
    "ADVANCES IN AGRONOMY, VOL 133",
    "ADVANCES IN AGRONOMY, VOL 134",
    "ADVANCES IN AGRONOMY, VOL 135",
    "ADVANCES IN AGRONOMY, VOL 136",
    "ADVANCES IN AGRONOMY, VOL 137",
    "ADVANCES IN AGRONOMY, VOL 138",
    "ADVANCES IN AGRONOMY, VOL 139",
    "AGRICULTURAL AND FOREST METEOROLOGY",
    "AGRONOMY FOR SUSTAINABLE DEVELOPMENT",
    "ANNUAL REVIEW OF APPLIED LINGUISTICS",
    "APPLIED LINGUISTICS",
    "ARCHIVES OF COMPUTATIONAL METHODS IN ENGINEERING",
    "BMJ QUALITY & SAFETY",
    "BRAIN AND LANGUAGE",
    "CHEMICAL REVIEWS" ,
    "CHEMICAL SOCIETY REVIEWS" ,
    "COGNITIVE LINGUISTICS" ,
    "COMBUSTION AND FLAME",
    "COMPOSITES PART B ENGINEERING",
    "COMPUTER AIDED CIVIL AND INFRASTRUCTURE ENGINEERING",
    "COMPUTER METHODS IN APPLIED MECHANICS AND ENGINEERING",
    "ENERGY & ENVIRONMENTAL SCIENCE" ,
    "GLOBAL CHANGE BIOLOGY BIOENERGY",
    "HEALTH AFFAIRS",
    "IEEE COMMUNICATIONS MAGAZINE",
    "IEEE COMMUNICATIONS SURVEYS AND TUTORIALS",
    "IEEE JOURNAL ON SELECTED AREAS IN COMMUNICATIONS",
    "IEEE NETWORK",
    "IEEE TRANSACTIONS ON INDUSTRIAL INFORMATICS",
    "IEEE TRANSACTIONS ON MEDICAL IMAGING",
    "IEEE WIRELESS COMMUNICATIONS",
    "INTERNATIONAL JOURNAL OF ENGINEERING SCIENCE",
    "JOURNAL OF MANAGEMENT" ,
    "JOURNAL OF MARKETING" ,
    "JOURNAL OF MEMORY AND LANGUAGE",
    "JOURNAL OF SECOND LANGUAGE WRITING",
    "JOURNAL OF STATISTICAL SOFTWARE",
    "JOURNAL OF THE ACADEMY OF MARKETING SCIENCE" ,
    "MEDICAL IMAGE ANALYSIS",
    "MILBANK QUARTERLY",
    "MODERN LANGUAGE JOURNAL",
    "NATURE CHEMISTRY" ,
    "NATURE PHYSICS" ,
    "PHYSICAL REVIEW X" ,
    "PHYSICS REPORTS REVIEW SECTION OF PHYSICS LETTERS" ,
    "QUALITY & SAFETY" ,
    "REPORTS ON PROGRESS IN PHYSICS" ,
    "REVIEW OF COGNITIVE LINGUISTICS" ,
    "REVIEWS OF MODERN PHYSICS" ,
    "THEORETICAL AND APPLIED GENETICS",
    "VALUE IN HEALTH"
    }

#Function that gets the Title of the abstract
def getTitle(title):
    txtFiles = list()
    for (dirpath, dirnames, filenames) in os.walk("../raw"):
        txtFiles += [os.path.join(dirpath, file) for file in filenames]

    for txtFile in txtFiles:
        if txtFile.endswith(".txt"):
            for line in list(open(txtFile)):
                if(line.startswith("TI ")):
                    nextline = list(open(txtFile))[list(open(txtFile)).index(line)+1];
                    if(not nextline.startswith("SO ")):
                        fileTitle = line.replace("TI ","").replace("\n"," ").replace("  "," ")+" ".join(nextline.split())
                    else:
                        fileTitle = line.replace("TI ","").replace("\n"," ")
                    if(fileTitle == title):
                        return fileTitle

#Functions that get the Journal based on a file name
def searchJournal(file1,file2):
    retorno = ""
    for j in getAllJournals():
        if(file1.find(j) != -1):
            retorno = file1
        elif(file2.find(j) != -1):
            retorno = file2
        elif( (file2.find(j) != -1) and (file1.find(j) != -1) ):
            retorno = "False"
    return retorno

def getJournal(file):
    file = file.split('.seg.json')[0]
    file1 = file.split('_')[-3].replace('-',' ').upper()
    file2 = file.split('_')[-2].replace('-',' ').upper()
    journal = searchJournal(file1,file2)
    return journal


#Function that gets Abstract's info (journal, field, discipline) from a json file 'file'
def getInfoFromFile(file):
            fields = dictionaries.dictionaryFields()
            disciplines = dictionaries.dictionaryDisciplines()
            journals = getAllJournals() 
            journal = ""
            
            journal = getJournal(file)    
            
            field = fields.get(journal)
            discipline = disciplines.get(field)
            abstract = dict()
            abstract['journal'] = journal
            abstract['field']   = field
            abstract['discipline'] = discipline
            return abstract


#Function that orders a dict with index:values by values
def sortDict(d):
    lista  = (sorted(d.items(), key = lambda x : x[1], reverse=True))
    retorno = dict()
    for value in lista:
        retorno[value[0]] = value[1]
    return retorno

#Function that gets the interval between the value start and the value end
def cutdict(pdict, start, length):
    counter = 0
    outdict = dict() 
    pdict = pdict.copy()
    labels = list(pdict.keys())
    for label in labels:
        if(counter>=start and counter < start+length):
            outdict[label] = pdict.get(label)
        counter += 1
    return outdict


#Function that creates the directories where charts will be saved
def createDirectories():
    if not os.path.exists('export'):
        os.mkdir('export')

    if not os.path.exists('export/field'):
        os.mkdir('export/field')

    if not os.path.exists('export/discipline'):
        os.mkdir('export/discipline')

    if not os.path.exists('export/journal'):
        os.mkdir('export/journal')

    if not os.path.exists('export/all'):
        os.mkdir('export/all')
        
    if not os.path.exists('export/field/frame'):
        os.mkdir('export/field/frame')

    if not os.path.exists('export/discipline/frame'):
        os.mkdir('export/discipline/frame')

    if not os.path.exists('export/journal/frame'):
        os.mkdir('export/journal/frame')

    if not os.path.exists('export/all/frame'):
        os.mkdir('export/all/frame')

    if not os.path.exists('export/field/lexical_unit'):
        os.mkdir('export/field/lexical_unit')

    if not os.path.exists('export/discipline/lexical_unit'):
        os.mkdir('export/discipline/lexical_unit')

    if not os.path.exists('export/journal/lexical_unit'):
        os.mkdir('export/journal/lexical_unit')

    if not os.path.exists('export/all/lexical_unit'):
        os.mkdir('export/all/lexical_unit')
        
# Function that plots charts
def generateChart(d, opt, info, optdir):
    
    dic = d
    
    #selecting only the 30 frames with more occurrence
    #dic = first30(dic)
    
    #setting the data that will appear in the chart
    labels = list(dic.keys())
    values = list(dic.values())
    index = np.arange(len(labels))
    
    #used to allow xaxis locators settings
    fig, ax = plt.subplots()
    
    #plotting bar chart
    ax.barh(index, values)
    
    #setting labels
    if(optdir == 'frame'):
        plt.ylabel('Frames', fontsize=5)
    elif(optdir == 'lexical_unit'):
        plt.ylabel('Lexical Units', fontsize=5)

    plt.xlabel('Occurrences', fontsize=5)
    
    plt.yticks(index, labels, fontsize=5, rotation=0)
    
    major = 10
    minor = 1
    bigger = max(values)
    if(bigger > 10):
        major = bigger/10
        major = math.ceil(major)
        while(major % 10 != 0):
            major = major+1
        minor = major/10
    else:
        major = 2
        minor = 1
    
    #configuring xtick Major and minor locators
    mlocatormax = MultipleLocator(major)
    mlocatormin = MultipleLocator(minor)
    
    ax.xaxis.set_major_locator(mlocatormax)
    ax.xaxis.set_major_formatter(FormatStrFormatter('%d'))
    ax.xaxis.set_minor_locator(mlocatormin)
    ax.xaxis.set_view_interval(0,bigger+major)
    
    #setting locators size
    ax.tick_params(axis='x', which='major', labelsize=6)
    ax.tick_params(axis='y', which='major', labelsize=5)
    
    if(info == 'all'):
        info = 'All Corpus'
        
    plt.title(info)
    
    if(optdir == 'frame'):
        plt.savefig('export/'+opt+'/frame/'+info+'.png', bbox_inches='tight', dpi=300)
    elif(optdir == 'lexical_unit'):
        plt.savefig('export/'+opt+'/lexical_unit/'+info+'.png', bbox_inches='tight', dpi=300)
        
    #plt.show()
    plt.close()
    
# function that counts the ocurrence of itens from a item_type in abstracts that match a category from a category_type (field,
# discipline, journal), adding this to a existing counting dict (itens_counted)
def countItens(item_type, itens_counted, *params):
    if params: 
        category = params[0]
        category_type = params[1]
    #traversing all files
    for file_name in files:            
        file_location = path+'/'+file_name
        abstract = getInfoFromFile(file_location)
        file_content = open(file_location,"r").readlines()

        #traversing all lines in each file (each line has a json with information about frames)
        for index, line in enumerate(file_content):
            if(index > 0):
                jsonline = json.loads(line)

                #verifying each item    
                for frame in jsonline.get('frames'):

                    if(item_type == 'frame'):
                        frame_name = frame.get('target').get('name').lower()
                        if(frame_name[len(frame_name)-1] == 's' or frame_name[len(frame_name)-4:len(frame_name)-1] == 'ies'):
                            frame_name =inflection.singularize(frame_name)
                        if params:
                            if(abstract.get(category_type) == category):
                                if(itens_counted.get(frame_name) != None):
                                    itens_counted[frame_name] += 1 
                                else:
                                    itens_counted[frame_name] = 1                                
                        else:
                                if(itens_counted.get(frame_name) != None):
                                    itens_counted[frame_name] += 1 
                                else:
                                    itens_counted[frame_name] = 1                     

                    if(item_type == 'lexical_unit'):
                        for span in frame.get('target').get('spans'):
                            lexical_unit = span.get('text').lower() 
                            if(lexical_unit[len(lexical_unit)-1] == 's' or lexical_unit[len(lexical_unit)-4:len(lexical_unit)-1] == 'ies'):
                                lexical_unit =inflection.singularize(lexical_unit) 
                            if params:
                                if(abstract.get(category_type) == category):   
                                    #se ja existir o index, soma
                                    if(itens_counted.get(lexical_unit) != None):
                                        itens_counted[lexical_unit] += 1 
                                    #senao cria o index sem modificacoes
                                    else:
                                        itens_counted[lexical_unit] = 1     
                            else:
                                if(itens_counted.get(lexical_unit) != None):
                                    itens_counted[lexical_unit] += 1 
                                else:
                                    itens_counted[lexical_unit] = 1 
    return itens_counted

# files = the files where the function should search for itens
# params = (optional) --- do not pass when considering all file
#       [0] the categories of abstract that the function will search in --- do not pass when considering all files
#       [1] the category type (field, discipline, journal)  

def itensPerCategory(files, item_type, *params):
    path = "../semafor_output" #files path
    mensagem = "Dados de: \n"  #log
    result = dict()            #output 
    
    #considering all files
    if not params: 
        print("Searching for "+item_type+"s in all files")
        all_result = dict()
        #all_result = first30(sortDict(countItens(item_type,all_result)))
        all_result = sortDict(countItens(item_type,all_result))
        qtd = len(all_result)
        cutstart = 0
        while(cutstart < qtd):
            generateChart(cutdict(all_result,cutstart,30), 'all', 'all ' + str(cutstart)+"_"+str(cutstart+30), item_type)
            cutstart = cutstart+30
        
        return all_result
    else:
        categories = params[0]
        category_type = params[1] 
        print("Searching for "+item_type+"s per "+category_type+"s")
        all_categories_result = dict()
        for category in categories:
            category_result = dict()
            #category_result = first30(sortDict(countItens(item_type, category_result, category, category_type)))
            category_result = sortDict(countItens(item_type, category_result, category, category_type))
            if(category_result):
                label = 'Lexical Units' if (item_type == 'lexical_unit') else 'Frames'
                qtd = len(category_result)
                cutstart = 0
                while(cutstart < qtd):
                    generateChart(cutdict(category_result,cutstart,30), category_type, category+" ("+label+") "+str(cutstart)+"_"+str(cutstart+30), item_type)
                    cutstart = cutstart+30
                all_categories_result[category] = category_result
            else:
                print("Nothing found for "+category_type+": "+category)
        return all_categories_result

#function that generates XLS files
def generateExcel(fpcat, opt, optdir):
    if(opt != "all"):
        for v in fpcat:
            vDict = fpcat.get(v)
            vDF = pd.DataFrame({optdir+'s':list(vDict.keys()),'occurrence':list(vDict.values())})
            vDF.to_excel("export/"+opt+"/"+optdir+"/"+v+".xlsx", index=False)
    else: 
        fpcat = pd.DataFrame({optdir+'s':list(fpcat.keys()),'occurrence':list(fpcat.values())})
        fpcat.to_excel("export/"+opt+"/"+optdir+"/All_Abstracts.xlsx", index=False)

In [3]:
# Selecting json files
path = "../semafor_output"
files = os.listdir(path)
cleanDirectory(files)

# Sorting files, disciplines and journals alphabetically
list_of_fields = list(getAllFields())
list_of_fields.sort()

list_of_disciplines = list(getAllDisciplines())
list_of_disciplines.sort()

list_of_journals = list(getAllJournals())
list_of_journals.sort()

# Creating the directories needed
createDirectories()

all_frames = itensPerCategory(files,'frame')
frames_per_field = itensPerCategory(files, 'frame', list_of_fields, 'field')
frames_per_discipline = itensPerCategory(files,'frame', list_of_disciplines, 'discipline')
frames_per_journal = itensPerCategory(files,'frame', list_of_journals, 'journal')

all_lexicalunits = itensPerCategory(files,'lexical_unit')
lexicalunits_per_field = itensPerCategory(files, 'lexical_unit', list_of_fields, 'field')
lexicalunits_per_discipline = itensPerCategory(files,'lexical_unit', list_of_disciplines, 'discipline')
lexicalunits_journal = itensPerCategory(files,'lexical_unit', list_of_journals, 'journal')

Searching for frames in all files
Searching for frames per fields
Searching for frames per disciplines
Searching for frames per journals
Nothing found for journal: ADVANCES IN AGRONOMY, VOL 136
Nothing found for journal: QUALITY & SAFETY
Searching for lexical_units in all files
Searching for lexical_units per fields
Searching for lexical_units per disciplines
Searching for lexical_units per journals
Nothing found for journal: ADVANCES IN AGRONOMY, VOL 136
Nothing found for journal: QUALITY & SAFETY


In [4]:
#xls files for all abstracts, by field, by discipline and by journal (frames)
generateExcel(all_frames,"all",'frame')
generateExcel(frames_per_field,"field",'frame')
generateExcel(frames_per_discipline,"discipline",'frame')
generateExcel(frames_per_journal,"journal",'frame')

#xls files for all abstracts, by field, by discipline and by journal (lexical units)
generateExcel(all_lexicalunits,"all",'lexical_unit')
generateExcel(lexicalunits_per_field,"field",'lexical_unit')
generateExcel(lexicalunits_per_discipline,"discipline",'lexical_unit')
generateExcel(lexicalunits_journal,"journal",'lexical_unit')


In [5]:
def generatePieChart(data,dir_filename):
    # Pie chart
    labels = data.keys()
    sizes = data.values()
    
    # only "explode" the 2nd slice (i.e. 'Hogs')
    explode = (0, 0.1, 0, 0)  
    fig1, ax1 = plt.subplots()
    ax1.pie(sizes, labels=labels, autopct='%1.1f%%', textprops={'size': 'smaller'}, pctdistance=1.1,
             labeldistance=1.3, startangle=90)
    
    # Equal aspect ratio ensures that pie is drawn as a circle
    ax1.axis('equal')  
    plt.tight_layout()
    fig = plt.gcf()
    fig.set_size_inches(10,10)  
    plt.savefig(dir_filename+'.png', bbox_inches='tight', dpi=300)
    #plt.show()
    plt.close()

In [6]:
f_total = sum(all_frames.values())
f_field_total = dict()
f_discipline_total = dict()
f_journal_total = dict()

lu_total = sum(all_lexicalunits.values())
lu_field_total = dict()
lu_discipline_total = dict()
lu_journal_total = dict()

for field in frames_per_field:
    f_field_total[field] = sum(frames_per_field.get(field).values()) 
for discipline in frames_per_discipline:
    f_discipline_total[discipline] = sum(frames_per_discipline.get(discipline).values())
for journal in frames_per_journal:
    f_journal_total[journal] = sum(frames_per_journal.get(journal).values())

for field in lexicalunits_per_field:
    lu_field_total[field] = sum(lexicalunits_per_field.get(field).values())
for discipline in lexicalunits_per_discipline:
    lu_discipline_total[discipline] = sum(lexicalunits_per_discipline.get(discipline).values())
for journal in lexicalunits_journal:
    lu_journal_total[journal] = sum(lexicalunits_journal.get(journal).values())
    
f_percen = dict()
f_field_percen = dict()
f_discipline_percen = dict()
f_journal_percen = dict()

lu_percen = dict()
lu_field_percen = dict()
lu_discipline_percen = dict()
lu_journal_percen = dict()

for frame in all_frames:
        f_percen[frame] = (all_frames.get(frame)/f_total)*100        
f_percen = sortDict(f_percen) 

for field in frames_per_field:
    f_field_percen[field] = dict()
    for frame in frames_per_field.get(field):
        f_field_percen[field][frame] = (frames_per_field.get(field).get(frame)/f_field_total.get(field))*100

for discipline in frames_per_discipline:
    f_discipline_percen[discipline] = dict()
    for frame in frames_per_discipline.get(discipline):
        f_discipline_percen[discipline][frame] = (frames_per_discipline.get(discipline).get(frame)/f_discipline_total.get(discipline))*100

for journal in frames_per_journal:
    f_journal_percen[journal] = dict()
    for frame in frames_per_journal.get(journal):
        f_journal_percen[journal][frame] = (frames_per_journal.get(journal).get(frame)/f_journal_total.get(journal))*100

for lexical_unit in all_lexicalunits:
        lu_percen[lexical_unit] = (all_lexicalunits.get(lexical_unit)/lu_total)*100        
lu_percen = sortDict(lu_percen) 

for field in lexicalunits_per_field:
    lu_field_percen[field] = dict()
    for lexical_unit in lexicalunits_per_field.get(field):
        lu_field_percen[field][lexical_unit] = (lexicalunits_per_field.get(field).get(lexical_unit)/lu_field_total.get(field))*100
        
for discipline in lexicalunits_per_discipline:
    lu_discipline_percen[discipline] = dict()
    for lexical_unit in lexicalunits_per_discipline.get(discipline):
        lu_discipline_percen[discipline][lexical_unit] = (lexicalunits_per_discipline.get(discipline).get(lexical_unit)/lu_discipline_total.get(discipline))*100

for journal in lexicalunits_journal:
    lu_journal_percen[journal] = dict()
    for lexical_unit in lexicalunits_journal.get(journal):
        lu_journal_percen[journal][lexical_unit] = (lexicalunits_journal.get(journal).get(lexical_unit)/lu_journal_total.get(journal))*100

def less_than_one(dict_percen):
    less_than_one_percent = 0;
    for percen in dict_percen.copy():
        if(dict_percen.get(percen) < 1):
            less_than_one_percent += dict_percen.get(percen)
            dict_percen.pop(percen)
    dict_percen['Less than one percent'] = less_than_one_percent
    return dict_percen

def more_than_one(dict_percen):
    for percen in dict_percen.copy():
        if(dict_percen.get(percen) < 1):
            dict_percen.pop(percen)
    return dict_percen

In [7]:
data = less_than_one(f_percen) 
generatePieChart(data,'export/'+'all'+'/'+'frame/'+'all'+'_lto')
data = more_than_one(f_percen) 
generatePieChart(data,'export/'+'all'+'/'+'frame/'+'all'+'_mto')

for field in f_field_percen:
    data = less_than_one(sortDict(f_field_percen.get(field)))
    generatePieChart(data,'export/'+'field'+'/'+'frame/'+field+'_lto')
    data = more_than_one(sortDict(f_field_percen.get(field)))
    generatePieChart(data,'export/'+'field'+'/'+'frame/'+field+'_mto')
    
for discipline in f_discipline_percen:
    data = less_than_one(sortDict(f_discipline_percen.get(discipline)))
    generatePieChart(data,'export/'+'discipline'+'/'+'frame/'+discipline+'_lto')
    data = more_than_one(sortDict(f_discipline_percen.get(discipline)))
    generatePieChart(data,'export/'+'discipline'+'/'+'frame/'+discipline+'_mto')
        
for journal in f_journal_percen:
    data = less_than_one(sortDict(f_journal_percen.get(journal)))
    generatePieChart(data,'export/'+'journal'+'/'+'frame/'+journal+'_lto')
    data = more_than_one(sortDict(f_journal_percen.get(journal)))
    generatePieChart(data,'export/'+'journal'+'/'+'frame/'+journal+'_mto') 
    


In [8]:
data = less_than_one(lu_percen) 
generatePieChart(data,'export/'+'all'+'/'+'lexical_unit/'+'all'+'_lto')
data = more_than_one(lu_percen) 
generatePieChart(data,'export/'+'all'+'/'+'lexical_unit/'+'all'+'_mto')

for field in lu_field_percen:
    data = less_than_one(sortDict(lu_field_percen.get(field)))
    generatePieChart(data,'export/'+'field'+'/'+'lexical_unit/'+field+'_lto')
    data = more_than_one(sortDict(lu_field_percen.get(field)))
    generatePieChart(data,'export/'+'field'+'/'+'lexical_unit/'+field+'_mto')
    
for discipline in lu_discipline_percen:
    data = less_than_one(sortDict(lu_discipline_percen.get(discipline)))
    generatePieChart(data,'export/'+'discipline'+'/'+'lexical_unit/'+discipline+'_lto')
    data = more_than_one(sortDict(lu_discipline_percen.get(discipline)))
    generatePieChart(data,'export/'+'discipline'+'/'+'lexical_unit/'+discipline+'_mto')
        
for journal in lu_journal_percen:
    data = less_than_one(sortDict(lu_journal_percen.get(journal)))
    generatePieChart(data,'export/'+'journal'+'/'+'lexical_unit/'+journal+'_lto')
    data = more_than_one(sortDict(lu_journal_percen.get(journal)))
    generatePieChart(data,'export/'+'journal'+'/'+'lexical_unit/'+journal+'_mto') 

In [9]:
data = lu_percen
for percen in data.copy():
    if(data.get(percen) < 0.5):
        data.pop(percen) 

generatePieChart(data,'export/'+'all'+'/'+'lexical_unit/'+'all'+'_mthalfpercent')

In [3]:
#dill.dump_session('notebook_env.db') #save
dill.load_session('notebook_env.db') #load

In [61]:
def countLexicalFrame(source, categ_type):
    counted = dict()
    for categ in source:
        categ_data = dict()
        if(categ_type == 'all'):
            categ_data = source
        else:
            categ_data = source.get(categ) 
        for frameindex in categ_data:
            #traversing all files
            for file_name in files:            
                file_location = path+'/'+file_name
                abstract = getInfoFromFile(file_location) 
                if(abstract.get(categ_type) == categ or categ_type == 'all'): 
                    file_content = open(file_location,"r").readlines()

                    #traversing all lines in each file (each line has a json with information about frames)
                    for index, line in enumerate(file_content):
                        if(index > 0):
                            jsonline = json.loads(line)  

                            #verifying each item    
                            for frame in jsonline.get('frames'):                            
                                frame_name = frame.get('target').get('name').lower()  
                                if(frame_name == frameindex): 
                                    for span in frame.get('target').get('spans'):
                                        lexical_unit = span.get('text').lower()  
                                        if(counted.get(categ) == None):  
                                            counted[categ] = dict()  
                                        if(counted.get(categ).get(frame_name) == None):   
                                            counted[categ][frame_name] = dict()
                                        if(counted.get(categ).get(frame_name).get(lexical_unit) == None):
                                            counted[categ][frame_name][lexical_unit] = 1
                                        else:
                                            counted[categ][frame_name][lexical_unit] += 1
                                        #print(counted)
    return counted

In [65]:
lu_per_frame_all = dict()
lu_per_frame_field = dict()
lu_per_frame_discipline = dict()
lu_per_frame_journal = dict()
                                    
lu_per_frame_all = countLexicalFrame(cutdict(all_frames,0,30),'all')
lu_per_frame_field = countLexicalFrame(cutdict(frames_per_field,0,30),'field')
lu_per_frame_discipline = countLexicalFrame(cutdict(frames_per_discipline,0,30),'discipline')
lu_per_frame_journal = countLexicalFrame(cutdict(frames_per_journal,0,30),'journal')
                                

In [2]:
#dill.dump_session('notebook_env.db') #save
dill.load_session('notebook_env.db') #load

In [ ]:
for field in lu_per_frame_field: 
    for frame in lu_per_frame_field.get(field):  
        generateChart(sortDict(lu_per_frame_field.get(field).get(frame)), 'field', 'LUs by Frame ('+field+' - '+frame+')', 'lexical_unit')   
    
for discipline in lu_per_frame_discipline:
    for frame in lu_per_frame_discipline.get(discipline): 
        generateChart(sortDict(lu_per_frame_discipline.get(discipline).get(frame)), 'discipline', 'LUs by Frame ('+discipline+' - '+frame+')', 'lexical_unit')   
    
for journal in lu_per_frame_journal:
    for frame in lu_per_frame_journal.get(journal): 
        generateChart(sortDict(lu_per_frame_journal.get(journal).get(frame)), 'journal', 'LUs by Frame ('+journal+' - '+frame+')', 'lexical_unit')   
    
for frame in lu_per_frame_all:
    generateChart(sortDict(lu_per_frame_all.get(frame)), 'all', 'LUs by Frame ('+'all'+' - '+frame+')', 'lexical_unit')   

        
    

In [38]:
f_percen = dict();
for frame in all_frames:
        f_percen[frame] = (all_frames.get(frame)/f_total)*100        
f_percen = sortDict(f_percen) 
#print(f_percen)

lu_percen = dict();
for lexical_unit in all_lexicalunits:
        lu_percen[lexical_unit] = (all_lexicalunits.get(lexical_unit)/lu_total)*100        
lu_percen = sortDict(lu_percen) 
#print(lu_percen)

In [33]:
def less_than_one(dict_percen):
    less_than_one_percent = 0;
    for percen in dict_percen.copy():
        if(dict_percen.get(percen) < 1):
            less_than_one_percent += dict_percen.get(percen)
            dict_percen.pop(percen)
    dict_percen['Less than one percent'] = less_than_one_percent
    return dict_percen

def more_than_one(dict_percen):
    for percen in dict_percen.copy():
        if(dict_percen.get(percen) < 1):
            dict_percen.pop(percen)
    return dict_percen

In [39]:
lu_lto = less_than_one(lu_percen.copy()) 
lu_mto = more_than_one(lu_percen.copy())
fr_lto = less_than_one(f_percen.copy()) 
fr_mto = more_than_one(f_percen.copy())
print(fr_lto)
print(fr_mto)
generatePieChart(fr_mto,'export/'+'all'+'/'+'frame/'+'all'+'_mto')

{'quantity': 2.596561386710224, 'scrutiny': 2.2445329991410508, 'mean': 2.100905417012828, 'cardinal_number': 2.0037455820437358, 'gizmo': 1.7038173958348002, 'statement': 1.6615739893264994, 'using': 1.4531731838855486, 'purpose': 1.4010729825253108, 'causation': 1.3982567554247574, 'temporal_collocation': 1.3208105101595393, 'aggregate': 1.2856076714026219, 'text': 1.2152019938887872, 'capability': 1.1574693383274426, 'likelihood': 1.1462044299252292, 'objective_influence': 1.096920455665545, 'increment': 1.0856555472633314, 'similarity': 1.0490445949561373, 'building': 1.02651477815171, 'Less than one percent': 73.05293098835548}
{'quantity': 2.596561386710224, 'scrutiny': 2.2445329991410508, 'mean': 2.100905417012828, 'cardinal_number': 2.0037455820437358, 'gizmo': 1.7038173958348002, 'statement': 1.6615739893264994, 'using': 1.4531731838855486, 'purpose': 1.4010729825253108, 'causation': 1.3982567554247574, 'temporal_collocation': 1.3208105101595393, 'aggregate': 1.285607671402621